In [12]:
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import sys
import numpy as np
import time

%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.externals import joblib
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier

sns.set_style('ticks')
sys.path.append("/home/mschlupp/pythonTools")

In [27]:
ga_data = pd.read_csv("files/finalSets/ga_noEvts.csv"
                      , usecols=["device_id","group"
                                , "phone_brand","isTrain"
                                , "device_model"])

## Encode the  colums 

In [29]:
enc_brand = LabelEncoder()
enc_device = LabelEncoder()
enc_group = LabelEncoder()

In [30]:
ga_data["phone_brand"] = enc_brand.fit_transform(ga_data.phone_brand)
ga_data["device_model"] = enc_device.fit_transform(ga_data.device_model)
ga_data["group"] = enc_group.fit_transform(ga_data.group)

In [39]:
drop_list = ["isTrain"]
train = ga_data[ga_data.isTrain==1].drop( drop_list, axis=1)
true_classes = train.group
test = ga_data[ga_data.isTrain==0].drop( drop_list+["group"], axis=1)

## We use the best single model from previous tests
XGB classifier 

In [14]:
# the parameters have been determined in the `predict_classe` notebook.
gbdt = GradientBoostingClassifier(loss='deviance'
                                  ,max_features=None
                                  , min_samples_leaf=800
                                  , learning_rate=0.005
                                  , n_estimators=700);

In [45]:
x_train, x_test, y_train, y_test = train_test_split(train.drop(["device_id","group"], axis=1)
                                                   , true_classes, test_size=0.3)

In [48]:
gbdt.fit(x_train, y_train)

GradientBoostingClassifier(init=None, learning_rate=0.005, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=800, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=700,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [52]:
probs_train = gbdt.predict_proba(x_train)
probs_test = gbdt.predict_proba(x_test)

In [54]:
print("Test MVA predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,probs_train))
print("Log loss on test set: ", log_loss(y_test,probs_test))

Test MVA predictions on test and training set:

Log loss on training set:  2.39388310435
Log loss on test set:  2.40801068014


Performs largely identical.

## compare with linear model

In [80]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2',C=1.0, tol=0.0001, solver='lbfgs', max_iter=600, multi_class='multinomial')

In [77]:
lr.fit(x_train,y_train)

LogisticRegression(C=2.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=600, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.1, verbose=0, warm_start=False)

In [78]:
probs_lr_train = lr.predict_proba(x_train)
probs_lr_test = lr.predict_proba(x_test)

In [79]:
print("Test MVA predictions on test and training set:\n")
print("Log loss on training set: ", log_loss(y_train,probs_lr_train))
print("Log loss on test set: ", log_loss(y_test,probs_lr_test))

Test MVA predictions on test and training set:

Log loss on training set:  2.42480967
Log loss on test set:  2.42941641365


# Save final xgb classifier

In [81]:
joblib.dump(gbdt, 'trainedModels/final_gbdt_goNoEvts.pkl', compress=3)

['trainedModels/final_gbdt_goNoEvts.pkl']

# Predict test sample and create submission file (part of it)

In [83]:
predictions = gbdt.predict_proba(test.drop(["device_id"], axis=1))

In [85]:
probas_gbdt_df = pd.DataFrame(predictions, index=test.device_id)
probas_gbdt_df.columns = enc_group.inverse_transform(probas_gbdt_df.columns)

In [88]:
probas_gbdt_df.to_csv("finalOutputs/gbdt_gaNoEvts_predictions.csv")